In [1]:
import pandas as pd

In [60]:
bxdf[['User ID', 'ISBN', 'Book-Rating']] = bxdf[0].str.split(';', expand=True)

KeyError: 0

In [85]:
import os
import requests
import zipfile
import io

def download_and_extract_zip(zip_url, target_directory):
    try:
        # Fetch the zip data from the URL
        response = requests.get(zip_url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx errors

        # Create the target directory if it doesn't exist
        if not os.path.exists(target_directory):
            os.makedirs(target_directory)

        # Load the zip data into a ZipFile object
        with zipfile.ZipFile(io.BytesIO(response.content), 'r') as z:
            # Extract all the files from the zip archive to the target directory
            z.extractall(target_directory)

        print("Download and extraction completed successfully.")
        return True

    except requests.exceptions.RequestException as e:
        print(f"Failed to download the zip file from {zip_url}. Error: {e}")
        return False
    except zipfile.BadZipFile as e:
        print(f"Failed to extract the zip file. Error: {e}")
        return False

# Usage example:
zip_url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip'
target_directory = 'data/raw/'
success = download_and_extract_zip(zip_url, target_directory)

if success:
    print(f"Files have been extracted to '{target_directory}'.")

Download and extraction completed successfully.
Files have been extracted to 'data/raw/'.


In [1]:
from scripts.download_data import DownloadData

In [2]:
dd = DownloadData()

In [3]:
dd.download_goodreads_csv()

File downloaded successfully: data/raw/goodreads/books.csv
File downloaded successfully: data/raw/goodreads/ratings.csv


In [8]:
dd.download_bookcrossing_csv()


Zip download and extraction completed successfully: data/raw/bookcrossing/


In [9]:
from scripts.process_data import ProcessData

In [10]:
proc_data = ProcessData()

In [75]:
pd.DataFrame()

""


In [11]:
df_gr_ratings, df_gr_books = proc_data.process_goodreads()
df_bx_ratings, df_bx_books, df_bx_users = proc_data.process_bookcrossings()

In [76]:
dfs = [df_bx_books,df_gr_ratings]

In [77]:
dfs[0]['ISBN']

0         0195153448
1         0002005018
2         0060973129
3         0374157065
4         0393045218
             ...    
271355    0440400988
271356    0525447644
271357    006008667X
271358    0192126040
271359    0767409752
Name: ISBN, Length: 271360, dtype: object

In [26]:
set1 = set(df_bx_books['ISBN'])
set2 = set(df_gr_ratings['isbn'])

# Find the common elements in both sets
common_elements = set1.intersection(set2)
next(iter(common_elements))

'067972477X'

In [34]:
def filter_rows_by_column_value(data, column_name, allowed_values):
    filtered_data = data[data[column_name].isin(allowed_values)]
    return filtered_data

In [38]:
bx_filtered = filter_rows_by_column_value(df_bx_books, 'ISBN' ,common_elements )
gr_filtered = filter_rows_by_column_value(df_gr_ratings, 'isbn' ,common_elements )

In [47]:
gr_filtered

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
10,11,77203,77203,3295919,283,1594480001,9.781594e+12,Khaled Hosseini,2003.0,The Kite Runner,...,1813044,1878095,59730,34288,59980,226062,628174,929591,https://images.gr-assets.com/books/1484565687m...,https://images.gr-assets.com/books/1484565687s...
17,18,5,5,2402163,376,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPré, Rufus Beck",1999.0,Harry Potter and the Prisoner of Azkaban,...,1832823,1969375,36099,6716,20413,166129,509447,1266670,https://images.gr-assets.com/books/1499277281m...,https://images.gr-assets.com/books/1499277281s...
59,60,1618,1618,4259809,183,1400032717,9.781400e+12,Mark Haddon,2003.0,The Curious Incident of the Dog in the Night-Time,...,867553,915790,35796,21049,58117,218218,355925,262481,https://images.gr-assets.com/books/1479863624m...,https://images.gr-assets.com/books/1479863624s...
74,75,227443,227443,3185154,193,014028009X,9.780140e+12,Helen Fielding,1996.0,NaN,...,736498,754498,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...
89,90,231804,231804,1426690,156,014038572X,9.780140e+12,S.E. Hinton,1967.0,The Outsiders,...,659248,680437,22662,10178,34346,134244,227250,274419,https://images.gr-assets.com/books/1442129426m...,https://images.gr-assets.com/books/1442129426s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9940,9941,61266,61266,59587,72,8807813025,9.788808e+12,Alessandro Baricco,1994.0,Novecento,...,9991,11676,575,121,448,1784,4052,5271,https://images.gr-assets.com/books/1346326152m...,https://images.gr-assets.com/books/1346326152s...
9951,9952,76248,76248,73768,16,051703817X,9.780517e+12,William S. Baring-Gould,1962.0,"Sherlock Holmes of Baker Street, the life of t...",...,8642,8719,36,101,169,1009,2269,5171,https://images.gr-assets.com/books/1355676228m...,https://images.gr-assets.com/books/1355676228s...
9961,9962,52588,52588,1135362,6,1580050751,9.781580e+12,"Inga Muscio, Betty Dodson",1998.0,Cunt: A Declaration of Independence,...,8203,8627,841,279,676,1864,2805,3003,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [78]:
df_bx_ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [66]:
import pandas as pd
merged_df = pd.merge(bx_filtered, df_bx_ratings, on="ISBN", how="inner")
merged_df['Book-Rating'] = merged_df['Book-Rating'].astype(int)
merged_df = merged_df[['ISBN','Book-Rating']]
average_values = merged_df.groupby("ISBN")["Book-Rating"].mean().reset_index()

In [67]:
merged_df.dtypes

ISBN           object
Book-Rating     int32
dtype: object

In [68]:
merged_df[merged_df['ISBN']=='006000150X']

,ISBN,Book-Rating
9490,006000150X,5
9491,006000150X,10


In [69]:
average_values

,ISBN,Book-Rating
0,006000150X,7.500000
1,006000942X,2.833333
2,006008216X,1.583333
3,006008460X,2.956522
4,006054094X,5.500000
...,...,...
392,843391426X,0.000000
393,8466302948,1.000000
394,8489163413,0.000000
395,8807813025,3.893617


In [72]:
merged_df2 = pd.merge(gr_filtered, average_values, left_on = "isbn", right_on="ISBN", how="inner")

In [85]:
merged_df2[['ISBN', 'authors','title','original_publication_year']]

,ISBN,authors,title,original_publication_year
0,1594480001,Khaled Hosseini,The Kite Runner,2003.0
1,043965548X,"J.K. Rowling, Mary GrandPré, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,1999.0
2,1400032717,Mark Haddon,The Curious Incident of the Dog in the Night-Time,2003.0
3,014028009X,Helen Fielding,"Bridget Jones's Diary (Bridget Jones, #1)",1996.0
4,014038572X,S.E. Hinton,The Outsiders,1967.0
...,...,...,...,...
392,8807813025,Alessandro Baricco,Novecento. Un monologo,1994.0
393,051703817X,William S. Baring-Gould,Sherlock Holmes of Baker Street: A Life of the...,1962.0
394,1580050751,"Inga Muscio, Betty Dodson",Cunt: A Declaration of Independence,1998.0
395,067973371X,Robert A. Caro,"Means of Ascent (The Years of Lyndon Johnson, #2)",1990.0


In [1]:
from scripts.process_data import ProcessData

In [2]:
proc_data = ProcessData()
proc_data.get_final_dataframe()

DataFrame saved to data/processed/reference_dataframe.pkl


,books_count,isbn,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,Book-Rating
0,283,1594480001,2003.0,4.26,1813044,1878095,59730,34288,59980,226062,628174,929591,2.400000
1,376,043965548X,1999.0,4.53,1832823,1969375,36099,6716,20413,166129,509447,1266670,3.533333
2,183,1400032717,2003.0,3.85,867553,915790,35796,21049,58117,218218,355925,262481,4.813187
3,193,014028009X,1996.0,3.75,736498,754498,8157,28973,59309,198555,254836,212825,3.751852
4,156,014038572X,1967.0,4.06,659248,680437,22662,10178,34346,134244,227250,274419,4.460674
...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,72,8807813025,1994.0,4.19,9991,11676,575,121,448,1784,4052,5271,3.893617
393,16,051703817X,1962.0,4.40,8642,8719,36,101,169,1009,2269,5171,5.000000
394,6,1580050751,1998.0,3.88,8203,8627,841,279,676,1864,2805,3003,3.238095
395,19,067973371X,1990.0,4.25,12582,12952,395,303,551,1737,3389,6972,0.000000


In [4]:
import pandas as pd
pd.read_pickle('data/processed/reference_dataframe.pkl')

,ISBN,authors,title,original_publication_year
0,1594480001,Khaled Hosseini,The Kite Runner,2003.0
1,043965548X,"J.K. Rowling, Mary GrandPré, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,1999.0
2,1400032717,Mark Haddon,The Curious Incident of the Dog in the Night-Time,2003.0
3,014028009X,Helen Fielding,"Bridget Jones's Diary (Bridget Jones, #1)",1996.0
4,014038572X,S.E. Hinton,The Outsiders,1967.0
...,...,...,...,...
392,8807813025,Alessandro Baricco,Novecento. Un monologo,1994.0
393,051703817X,William S. Baring-Gould,Sherlock Holmes of Baker Street: A Life of the...,1962.0
394,1580050751,"Inga Muscio, Betty Dodson",Cunt: A Declaration of Independence,1998.0
395,067973371X,Robert A. Caro,"Means of Ascent (The Years of Lyndon Johnson, #2)",1990.0


In [82]:
columns_to_drop = [
    'authors',
    'best_book_id',
    'book_id',
    'goodreads_book_id',
    'image_url',
    'isbn13',
    'ISBN',
    'language_code',
    'original_title',
    'small_image_url',
    'title',
    'work_id'
]

merged_df2.drop(columns = columns_to_drop)

,books_count,isbn,original_publication_year,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,Book-Rating
0,283,1594480001,2003.0,4.26,1813044,1878095,59730,34288,59980,226062,628174,929591,2.400000
1,376,043965548X,1999.0,4.53,1832823,1969375,36099,6716,20413,166129,509447,1266670,3.533333
2,183,1400032717,2003.0,3.85,867553,915790,35796,21049,58117,218218,355925,262481,4.813187
3,193,014028009X,1996.0,3.75,736498,754498,8157,28973,59309,198555,254836,212825,3.751852
4,156,014038572X,1967.0,4.06,659248,680437,22662,10178,34346,134244,227250,274419,4.460674
...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,72,8807813025,1994.0,4.19,9991,11676,575,121,448,1784,4052,5271,3.893617
393,16,051703817X,1962.0,4.40,8642,8719,36,101,169,1009,2269,5171,5.000000
394,6,1580050751,1998.0,3.88,8203,8627,841,279,676,1864,2805,3003,3.238095
395,19,067973371X,1990.0,4.25,12582,12952,395,303,551,1737,3389,6972,0.000000


In [35]:
filter_rows_by_column_value(df_gr_ratings, 'isbn' ,common_elements )

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
10,11,77203,77203,3295919,283,1594480001,9.781594e+12,Khaled Hosseini,2003.0,The Kite Runner,...,1813044,1878095,59730,34288,59980,226062,628174,929591,https://images.gr-assets.com/books/1484565687m...,https://images.gr-assets.com/books/1484565687s...
17,18,5,5,2402163,376,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPré, Rufus Beck",1999.0,Harry Potter and the Prisoner of Azkaban,...,1832823,1969375,36099,6716,20413,166129,509447,1266670,https://images.gr-assets.com/books/1499277281m...,https://images.gr-assets.com/books/1499277281s...
59,60,1618,1618,4259809,183,1400032717,9.781400e+12,Mark Haddon,2003.0,The Curious Incident of the Dog in the Night-Time,...,867553,915790,35796,21049,58117,218218,355925,262481,https://images.gr-assets.com/books/1479863624m...,https://images.gr-assets.com/books/1479863624s...
74,75,227443,227443,3185154,193,014028009X,9.780140e+12,Helen Fielding,1996.0,NaN,...,736498,754498,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...
89,90,231804,231804,1426690,156,014038572X,9.780140e+12,S.E. Hinton,1967.0,The Outsiders,...,659248,680437,22662,10178,34346,134244,227250,274419,https://images.gr-assets.com/books/1442129426m...,https://images.gr-assets.com/books/1442129426s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9940,9941,61266,61266,59587,72,8807813025,9.788808e+12,Alessandro Baricco,1994.0,Novecento,...,9991,11676,575,121,448,1784,4052,5271,https://images.gr-assets.com/books/1346326152m...,https://images.gr-assets.com/books/1346326152s...
9951,9952,76248,76248,73768,16,051703817X,9.780517e+12,William S. Baring-Gould,1962.0,"Sherlock Holmes of Baker Street, the life of t...",...,8642,8719,36,101,169,1009,2269,5171,https://images.gr-assets.com/books/1355676228m...,https://images.gr-assets.com/books/1355676228s...
9961,9962,52588,52588,1135362,6,1580050751,9.781580e+12,"Inga Muscio, Betty Dodson",1998.0,Cunt: A Declaration of Independence,...,8203,8627,841,279,676,1864,2805,3003,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [29]:
df_gr_ratings[df_gr_ratings['isbn']=='067972477X']

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
2797,2798,18765,18765,4232388,152,067972477X,9.780680e+12,Robert Graves,1934.0,"I, Claudius",...,38484,41903,1862,425,1092,5360,14800,20226,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [27]:
df_bx_books[df_bx_books['ISBN']=='067972477X']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
43820,067972477X,"I, Claudius : From the Autobiography of Tiberi...",Robert Graves,1989,Vintage Books USA,http://images.amazon.com/images/P/067972477X.0...,http://images.amazon.com/images/P/067972477X.0...,http://images.amazon.com/images/P/067972477X.0...


In [16]:
df_bx_books['ISBN'].intersection(df_gr_ratings['isbn'])

AttributeError: 'Series' object has no attribute 'intersection'

In [15]:
df_bx_books['ISBN']

0         0195153448
1         0002005018
2         0060973129
3         0374157065
4         0393045218
             ...    
271355    0440400988
271356    0525447644
271357    006008667X
271358    0192126040
271359    0767409752
Name: ISBN, Length: 271360, dtype: object

In [20]:
df_bx_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
import pandas as pd

def get_csv_header(file_path):
    """
    Get the header (column names) from a CSV file, removing quotations and splitting by ';'.

    Parameters:
        file_path (str): The path to the CSV file.

    Returns:
        list: A list containing the cleaned column names (header) of the CSV file.
              Returns an empty list if the file is empty or cannot be read.
    """
    try:
        df = pd.read_csv(file_path, encoding='unicode_escape', nrows=0, sep=';')  # Read only the first row (header) using ';' as separator
        header = list(df.columns)
        cleaned_header = [col.strip('"') for col in header]  # Remove quotations from column names
        return cleaned_header
    except pd.errors.EmptyDataError:
        print(f"The CSV file at {file_path} is empty.")
        return []
    except pd.errors.ParserError as e:
        print(f"Failed to read the CSV file at {file_path}. Error: {e}")
        return []

# Example usage:
csv_file_path = 'data/raw/bookcrossing/BX-Books.csv'
header = get_csv_header(csv_file_path)
print("CSV Header:", header)


CSV Header: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']


In [7]:
import pandas as pd
df = pd.read_csv('data/raw/bookcrossing/BX-Book-Ratings.csv', encoding='unicode_escape', on_bad_lines='skip',sep=';')  # Read only the first row (header) using ';' as separator


In [8]:
df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
|